<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/LEJEPA_VJEPA_AGI_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/rbalestr-lab/lejepa/tree/main

1. World models that can predict and reason about real situations, not just text (LeCun’s Joint Embedding Predictive Architecture, 2022).
2. Autonomous learning that discovers causal structure instead of memorizing patterns.
3. Energy-based or modular systems that reason, plan, and act coherently within physical and ethical boundaries.
4. Embodied sentience and salience — systems grounded in sensory experience, capable of focusing on what truly matters and aligning ethically with human values.
5. Cognitive world models and evolutionary learning modules — hybrid systems that combine:
• Common-sense reasoning about space, time, and agency,
• Evolutionary and meta-learning algorithms that improve over generations of experience, and
• Analog–digital integration layers that bridge symbolic reasoning with continuous perception.

In [1]:
!pip install av -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 56.1 MB/s eta 0:00:00


In [2]:
!nvidia-smi

Thu Nov 13 21:20:40 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:00:05.0 Off |                    0 |
| N/A   30C    P0             50W /  400W |       0MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!df -h /content/gdrive

In [ ]:
!git clone https://github.com/castacks/TartanAviation.git
%cd TartanAviation

In [ ]:
%cd /content

In [ ]:
!pip install minio boto3 -q
!apt-get install -y unzip ffmpeg

## TartanAviation-vision

In [8]:
!cd /content/TartanAviation/vision

In [ ]:
!python /content/TartanAviation/vision/download.py --save_dir /content/gdrive/MyDrive/datasets/TartanAviation/vision --option Sample

In [ ]:
!python /content/TartanAviation/vision/download.py --save_dir /content/gdrive/MyDrive/datasets/TartanAviation/vision --option Sample --extract_frames

In [ ]:
!ls -lh /content/gdrive/MyDrive/datasets/TartanAviation/vision/
!ls -lh /content/gdrive/MyDrive/datasets/TartanAviation/vision/1_2023-02-22-15-21-49/

## TartanAviation-adsb

In [12]:
!ls -lh /content/gdrive/MyDrive/datasets/TartanAviation/adsb/kbtp/raw/2022/

total 701M
-rw------- 1 root root 700M Oct 20 11:25 2022.zip
-rw------- 1 root root 1.4M Nov 13 20:47 classifier_head_trained_on_tartan_aviation_sample.pth


In [13]:
!mkdir -p /content/adsb/kbtp/raw/2022

In [ ]:
%cd /content/gdrive/MyDrive/datasets/TartanAviation/adsb/kbtp/raw/2022
!unzip 2022.zip -d /content/adsb/kbtp/raw/2022/

In [15]:
# Please install OpenAI SDK first: `pip3 install openai`

from openai import OpenAI

from google.colab import userdata

api_key=userdata.get("DEEPSEEK_API_KEY")

client = OpenAI(api_key=api_key, base_url="https://api.deepseek.com")

## CELL1

In [ ]:
# Cell 1: Conceptual Modifications - Aviation Data Definitions

import torch
import numpy as np
import os
import glob
import av
import json
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoVideoProcessor, AutoModel
from tqdm.auto import tqdm
import logging
import datetime
import pytz
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s')



class AgentConfig:
    LLM_MODEL_NAME: str = "deepseek-reasoner"
    CLASS_LABELS = [
        "airplane landing",
        "airplane takeoff",
        "airport ground operations",
        "in-flight cruise",
        "emergency landing",
        "pre-flight check/maintenance",
        "en-route cruise",
        "climb phase",
        "descent phase",
        "holding pattern"
    ]

# Define num_classes globally
num_classes = len(AgentConfig.CLASS_LABELS)

# --- FIX: CLASSIFIER_SAVE_PATH moved to global scope ---
CLASSIFIER_SAVE_PATH = "classifier_head_trained_on_tartan_aviation_sample.pth"

AIRPORTS = {
    "CYUL": {"name": "Montreal-Trudeau International", "lat": 45.4706, "lon": -73.7408, "elevation_ft": 118},
    "LFPG": {"name": "Paris-Charles de Gaulle", "lat": 49.0097, "lon": 2.5479, "elevation_ft": 392},
}

AIRCRAFT_PERFORMANCE = {
    "Boeing777_300ER": {
        "cruise_speed_knots": 490,
        "fuel_burn_kg_per_hour": 7000,
        "max_range_nm": 7900,
        "climb_rate_fpm": 2500,
        "descent_rate_fpm": 2000,
        "typical_cruise_altitude_ft": 37000,
        "fuel_capacity_kg": 145000
    }
}

hf_repo = "facebook/vjepa2-vitg-fpc64-256"
EXTRACTED_FEATURES_DIR = "/content/gdrive/MyDrive/datasets/TartanAviation_VJEPA_Features/"

TOTAL_FLATTENED_VJEPA_DIM = 2048 * 1408

CONCEPTUAL_PLDM_LATENT_DIM = 1024

latent_dim_pldm = CONCEPTUAL_PLDM_LATENT_DIM
action_dim = 8

def load_and_process_video(video_path, processor_instance, model_instance, device_instance, num_frames_to_sample=16):
    """
    Loads a video, samples frames, and extracts V-JEPA features.
    Returns extracted features (torch.Tensor, shape like [1, 2048, 1408]) and the frames.
    Does NOT flatten the V-JEPA output here, keeping it as model's raw output.
    """
    frames = []
    if not os.path.exists(video_path):
        logging.error(f"ERROR: Video file '{video_path}' not found.")
        return None, None
    try:
        container = av.open(video_path)
        total_frames_in_video = container.streams.video[0].frames
        sampling_interval = max(1, total_frames_in_video // num_frames_to_sample)
        logging.info(f"Total frames in video: {total_frames_in_video}")
        logging.info(f"Sampling interval: {sampling_interval} frames")

        for i, frame in enumerate(container.decode(video=0)):
            if len(frames) >= num_frames_to_sample:
                break
            if i % sampling_interval == 0:
                img = frame.to_rgb().to_ndarray()
                frames.append(img)

        if not frames:
            logging.error(f"ERROR: No frames could be loaded from '{video_path}'.")
            return None, None
        elif len(frames) < num_frames_to_sample:
            logging.warning(f"WARNING: Only {len(frames)} frames loaded. Requested: {num_frames_to_sample}.")

        inputs = processor_instance(videos=list(frames), return_tensors="pt")
        inputs = {k: v.to(device_instance) for k, v in inputs.items()}

        with torch.no_grad():
            features = model_instance(**inputs).last_hidden_state

        logging.info(f"Successfully extracted V-JEPA features with raw shape: {features.shape}")
        return features, frames

    except av.FFmpegError as e:
        logging.error(f"Error loading video with PyAV: {e}")
        logging.error("This might indicate an issue with the video file itself or PyAV installation.")
        return None, None
    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")
        logging.error("Ensure 'av' library is installed (`pip install av`) and video file is not corrupt.")
        return None, None

class ClassifierHead(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        return self.fc2(self.dropout(self.relu(self.fc1(x))))

class LatentDynamicsPredictor(torch.nn.Module):
    def __init__(self, latent_dim, action_dim):
        super().__init__()
        self.layers = torch.nn.Sequential(
            nn.Linear(latent_dim + action_dim, 256),
            nn.ReLU(),
            nn.Linear(256, latent_dim)
        )

    def forward(self, latent_state, action):
        combined_input = torch.cat([latent_state, action], dim=-1)
        predicted_next_latent_state = self.layers(combined_input)
        return predicted_next_latent_state

class LatentProjector_old(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.projector = nn.Linear(input_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.relu(self.projector(x))


class LatentProjector(nn.Module):
    def __init__(self, input_dim=4, output_dim=1024):
        super().__init__()
        self.projector = nn.Linear(input_dim, output_dim)
        self.relu = nn.ReLU()



print("\n--- Instantiating Models and Optimizers ---")
model = AutoModel.from_pretrained(hf_repo)
processor = AutoVideoProcessor.from_pretrained(hf_repo)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

latent_projector = LatentProjector(TOTAL_FLATTENED_VJEPA_DIM, CONCEPTUAL_PLDM_LATENT_DIM)
latent_projector.to(device)

predictor = LatentDynamicsPredictor(latent_dim_pldm, action_dim)
predictor.to(device)
optimizer_pldm = torch.optim.Adam(list(predictor.parameters()) + list(latent_projector.parameters()), lr=0.001)

classifier = ClassifierHead(input_dim=1408, num_classes=num_classes)
classifier.to(device)

print(f"Models instantiated and moved to {device}.")
print("\nCell 1 setup complete for conceptual flight planning.")

## CELL2

In [ ]:
#Cell 2: Core Execution Feature Extraction, Classifier Training & Inference, LLM Interaction, and PLDM Training/Planning
# This cell assumes Cell 1 has been successfully executed in the current session.
# All objects (model, processor, classifier, predictor, device, optimizer_pldm)
# and all function definitions (load_and_process_video, ClassifierHead, LatentDynamicsPredictor)
# are expected to be available from Cell 1's execution.
import os
import logging
import torch
import json
from google.colab import drive
from tqdm.auto import tqdm
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import datetime
import pytz

#Mounting Google Drive
print("\n--- Cell 2: Mounting Google Drive for dataset access ---")
drive.mount('/content/gdrive')
print("Google Drive mounted.")

print(f"Checking for extracted features directory: {EXTRACTED_FEATURES_DIR}")
if not os.path.exists(EXTRACTED_FEATURES_DIR):
    logging.error(f"ERROR: Extracted features directory '{EXTRACTED_FEATURES_DIR}' not found. Please create it and upload V-JEPA features.")
    exit()
else:
    print(f"Extracted features directory found at {EXTRACTED_FEATURES_DIR}")

# Part 1: Load and process airplane-landing.mp4 for initial observation
print(f"\n--- Cell 2: Part 1 - Loading actual video '/content/gdrive/MyDrive/datasets/TartanAviation_VJEPA_Features/airplane-landing.mp4' for feature extraction ---")
flight_video_path = '/content/gdrive/MyDrive/datasets/TartanAviation_VJEPA_Features/airplane-landing.mp4'
# Use the defined load_and_process_video helper function. It now returns features and frames.
video_features_for_inference_raw, frames_for_pldm_planning = load_and_process_video(flight_video_path, processor, model, device_instance=device)

# -- CRITICAL: Process raw V-JEPA features to match ClassifierHead's expected input_dim --
if video_features_for_inference_raw is not None:
    # V-JEPA output shape is typically [1, 2048, 1408] (Batch, Channels, Height * Width if 1D)
    # Your old code pooled it as squeeze(0).mean(dim=0).unsqueeze(0), which resulted in [1, 1408] for classifier.
    # So, extracted_embedding_dim should be 1408 for the classifier.
    pooled_features_for_classifier = video_features_for_inference_raw.squeeze(0).mean(dim=0).unsqueeze(0)
    extracted_embedding_dim_for_classifier = pooled_features_for_classifier.shape[-1]
    logging.info(f"Dynamically determined extracted_embedding_dim for ClassifierHead: {extracted_embedding_dim_for_classifier}")
else:
    pooled_features_for_classifier = None
    extracted_embedding_dim_for_classifier = -1
    logging.error("Failed to extract video features for classifier. Exiting Cell 2.")
    exit()

# Part 2: Classifier Training
print(f"\n--- Cell 2: Part 2 - Starting Classifier Training ---")
print(f"Attempting to load real V-JEPA features for classifier training or generate synthetic data.")
print(f"Using device for classifier training: {device}")

try:
    # Re-initialize classifier with the correct, dynamically determined input_dim
    classifier = ClassifierHead(input_dim=extracted_embedding_dim_for_classifier, num_classes=num_classes)
    classifier.to(device)

    train_features_list = []
    train_labels_list = []

    map_file_path = os.path.join(EXTRACTED_FEATURES_DIR, "feature_label_map.json")
    if not os.path.exists(map_file_path):
        logging.warning(f"Feature-label map file '{map_file_path}' not found. Generating synthetic data.")
        feature_label_map = {}
    else:
        with open(map_file_path, 'r') as f:
            feature_label_map = json.load(f)

    if not feature_label_map:
        logging.warning(f"Feature-label map at {map_file_path} is empty. Generating synthetic data.")
        num_training_samples = 2_000_000
        # Synthetic data generation uses the dynamically determined input_dim
        train_features = torch.rand(num_training_samples, extracted_embedding_dim_for_classifier)
        train_labels = torch.randint(0, num_classes, (num_training_samples,))
        train_loader = DataLoader(TensorDataset(train_features, train_labels), batch_size=32, shuffle=True)
        val_loader = None
        print(f"Loaded {num_training_samples} SYNTHETIC features for training.")
    else:
        for item in tqdm(feature_label_map, desc="Loading real V-JEPA features"):
            feature_path = item['feature_path']
            label_idx = item['label_idx']
            try:
                if not os.path.isabs(feature_path):
                    feature_path = os.path.join(EXTRACTED_FEATURES_DIR, feature_path)

                if not os.path.exists(feature_path):
                    logging.warning(f"Feature file not found at {feature_path}. Skipping.")
                    continue

                feature = torch.load(feature_path, map_location=device)

                # Match your working code's pooling/squashing logic to get [1408] dim
                if feature.ndim == 3:
                    feature = feature.squeeze(0).mean(dim=0)
                elif feature.ndim == 2:
                    if feature.shape[0] == 1 and feature.shape[1] == 1408:
                        feature = feature.squeeze(0)
                    elif feature.shape[1] == 1408:
                        feature = feature.mean(dim=0)
                    else:
                        feature = feature.flatten()
                elif feature.ndim == 1:
                    pass
                else:
                    logging.warning(f"Skipping malformed feature from {feature_path}. Unexpected dimensions: {feature.ndim}")
                    continue

                # Final check after processing. Should be 1D with 1408 elements.
                if feature.shape[0] != extracted_embedding_dim_for_classifier:
                    logging.warning(f"Skipping feature at {feature_path}. Dimension mismatch: expected {extracted_embedding_dim_for_classifier}, got {feature.shape[0]}.")
                    continue

                train_features_list.append(feature)
                train_labels_list.append(label_idx)

            except Exception as e:
                logging.error(f"Error loading feature from {feature_path}: {e}. Skipping.")

        if train_features_list:
            train_features = torch.stack(train_features_list).to(device)
            train_labels = torch.tensor(train_labels_list).to(device)
            num_training_samples = len(train_features)
            print(f"Loaded {num_training_samples} REAL V-JEPA features for training.")

            if num_training_samples < 2:
                print("WARNING: Only 1 real V-JEPA feature loaded. Training may be unstable. Consider more data.")
                train_loader = DataLoader(TensorDataset(train_features, train_labels), batch_size=1, shuffle=True)
                val_loader = None
            else:
                dataset_size = len(train_features)
                train_size = int(0.8 * dataset_size)
                val_size = dataset_size - train_size

                if val_size == 0 and train_size > 0:
                    train_size = dataset_size
                    train_dataset_real = TensorDataset(train_features, train_labels)
                    val_dataset_real = None
                else:
                    train_dataset_real, val_dataset_real = torch.utils.data.random_split(
                        TensorDataset(train_features, train_labels), [train_size, val_size]
                    )
                train_loader = DataLoader(train_dataset_real, batch_size=32, shuffle=True)
                val_loader = DataLoader(val_dataset_real, batch_size=32, shuffle=False) if val_dataset_real else None
                print(f"Training on {len(train_dataset_real)} samples, Validation on {len(val_dataset_real) if val_dataset_real else 0} samples.")
        else:
            logging.error("No real V-JEPA features could be loaded from map file. Generating synthetic data as fallback.")
            num_training_samples = 2_000_000
            train_features = torch.rand(num_training_samples, extracted_embedding_dim_for_classifier)
            train_labels = torch.randint(0, num_classes, (num_training_samples,))
            train_loader = DataLoader(TensorDataset(train_features, train_labels), batch_size=32, shuffle=True)
            val_loader = None
            print(f"Loaded {num_training_samples} SYNTHETIC features for training as fallback.")

    criterion = torch.nn.CrossEntropyLoss()
    optimizer_classifier = torch.optim.Adam(classifier.parameters(), lr=0.001)

    num_epochs = 20
    for epoch in range(num_epochs):
        classifier.train()
        running_loss = 0.0
        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer_classifier.zero_grad()
            outputs = classifier(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer_classifier.step()
            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader.dataset)

        val_loss = 0.0
        if val_loader and len(val_loader.dataset) > 0:
            classifier.eval()
            with torch.no_grad():
                for inputs, labels in val_loader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = classifier(inputs)
                    loss = criterion(outputs, labels)
                    val_loss += loss.item()
            val_loss /= len(val_loader.dataset)
            logging.info(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}")
        else:
            logging.info(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f}")
            print("No validation data available or validation dataset is empty.")


    print("\n--- Classifier Training Complete ---")
    torch.save(classifier.state_dict(), CLASSIFIER_SAVE_PATH)
    print(f"Classifier saved to: {CLASSIFIER_SAVE_PATH}")
except Exception as e:
    logging.error(f"Error during classifier training: {e}")


print("Cell 2 execution complete.")

In [18]:
# Import OpenAI client for DeepSeek API
from openai import OpenAI
from google.colab import userdata

# --- DeepSeek API Setup ---
api_key = userdata.get("DEEPSEEK_API_KEY")

if not api_key:
    print("Error: DEEPSEEK_API_KEY not found in userdata.")
    print("Please set your DeepSeek API key in Colab secrets.")
    exit()

client = OpenAI(api_key=api_key, base_url="https://api.deepseek.com")
MODEL = "deepseek-reasoner"

In [19]:

#Part 3: Classification Inference and DEEPSEEK LLM Interaction
print("\n--- Cell 2: Part 3 - Starting V-JEPA Feature-Driven Classification Inference and DEEPSEEK LLM Interaction ---")
if pooled_features_for_classifier is None:
    logging.error("ERROR: Cannot perform classifier inference as 'pooled_features_for_classifier' is None.")
else:
    try:
        pooled_features_for_inference_on_device = pooled_features_for_classifier.to(device)

        classifier.load_state_dict(torch.load(CLASSIFIER_SAVE_PATH, map_location=device))
        logging.info(f"Classifier weights loaded from: {CLASSIFIER_SAVE_PATH}")

        classifier.eval()
        with torch.no_grad():
            logits = classifier(pooled_features_for_inference_on_device)
            probabilities = torch.softmax(logits, dim=1)

        predicted_class_idx = torch.argmax(probabilities, dim=1).item()
        predicted_confidence = probabilities[0, predicted_class_idx].item()


        # --- FIX: Use AgentConfig.CLASS_LABELS ---
        predicted_label = AgentConfig.CLASS_LABELS[predicted_class_idx]

        llm_input_description = ""
        if predicted_label == "airplane landing":
            llm_input_description = "The visual system detected an airplane landing."
        elif predicted_label == "airplane takeoff":
            llm_input_description = "The visual system detected an airplane takeoff."
        elif predicted_label == "airport ground operations":
            llm_input_description = "The visual system detected airport ground operations."
        elif predicted_label == "in-flight cruise":
            llm_input_description = "The visual system detected an airplane in-flight cruise."
        elif predicted_label == "emergency landing":
            llm_input_description = "The visual system detected a possible emergency landing."
        elif predicted_label == "pre-flight check/maintenance":
            llm_input_description = "The visual system detected pre-flight check or maintenance activity."
        else:
            llm_input_description = "The visual system detected an unrecognised flight activity."

        llm_input_description += f" (Confidence: {predicted_confidence:.2f})"

        print(f"\n--- AI Agent's Understanding from Classifier ---")
        print(f"**Primary Classification (Predicted by AI):** '{predicted_label}' {llm_input_description.split('Confidence:')[1].strip()}")
        print(f"**Description for LLM:** {llm_input_description}")
        print(f"Note: This classification's accuracy depends heavily on the quality and size of the real dataset used for classifier training.")

        print("\n--- Engaging DEEPSEEK LLM for Further Reasoning ---")
        try:
            llm_model = AgentConfig.LLM_MODEL_NAME

            prompt_for_deepseek = f"""
                  You are an AI assistant for flight planning operations.
                  Current visual observation: {llm_input_description}
                  Current time (EST): {datetime.datetime.now(pytz.timezone('EST')).strftime('%Y-%m-%d %H:%M:%S EST')}

                  Based on this visual observation, provide a concise operational assessment relevant to flight planning.
                  If the observation seems random or uncertain, state that. Do not add any conversational filler.
                  """


            deepseek_response = client.chat.completions.create(
            model=llm_model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant"},
                {"role": "user", "content": prompt_for_deepseek},
            ],
            stream=False
            )


            print("\n--- DEEPSEEK LLM Response ---")
            if deepseek_response.choices and deepseek_response.choices[0].message.content:
                print(deepseek_response.choices[0].message.content)
                print("--- DEEPSEEK LLM Response - END ---")
                print('\n')
            else:

                print("DEEPSEEK LLM did not provide a text response or cannot provide one.")
                # Check if there's an error attribute before trying to print it
                if hasattr(deepseek_response, 'error') and deepseek_response.error:
                     print(f"LLM Error: {deepseek_response.error}")
                # Check if there's a prompt_feedback attribute before trying to print it
                if hasattr(deepseek_response, 'prompt_feedback') and deepseek_response.prompt_feedback:
                    print(f"Prompt Feedback: {deepseek_response.prompt_feedback}")


        except Exception as llm_e:
            logging.error(f"Error interacting with DEEPSEEK LLM: {llm_e}")
            logging.error("Ensure your DEEPSEEK_API_KEY is correctly set in Colab Secrets.")

    except Exception as e:
        logging.error(f"Error during classification inference or overall Cell 2 execution: {e}")

print(f"The V-JEPA features (shape: {pooled_features_for_classifier.shape}) are the core input that a trained classifier would learn from.")
print(f"Current time in EST: {datetime.datetime.now(pytz.timezone('EST')).strftime('%Y-%m-%d %H:%M:%S EST')}")

print("\nCell 2 execution complete.")


--- Cell 2: Part 3 - Starting V-JEPA Feature-Driven Classification Inference and DEEPSEEK LLM Interaction ---

--- AI Agent's Understanding from Classifier ---
**Primary Classification (Predicted by AI):** 'airplane landing' 1.00)
**Description for LLM:** The visual system detected an airplane landing. (Confidence: 1.00)
Note: This classification's accuracy depends heavily on the quality and size of the real dataset used for classifier training.

--- Engaging DEEPSEEK LLM for Further Reasoning ---

--- DEEPSEEK LLM Response ---
Active landing confirms runway occupancy; assess for potential impacts on arrival sequencing and departure schedules.
--- DEEPSEEK LLM Response - END ---


The V-JEPA features (shape: torch.Size([1, 1408])) are the core input that a trained classifier would learn from.
Current time in EST: 2025-11-13 16:23:49 EST

Cell 2 execution complete.


## CELL3-NEW



LEJEPA, or the **L**ean **J**oint-**E**mbedding **P**redictive **A**rchitecture, is fundamentally defined by a **new loss calculation** and a set of architectural principles.

Here is a breakdown of why this loss is significant:

* ### **1. The Core Loss Components**
    The total LEJEPA loss is a combination of two essential parts:
    * **JEPA Predictive Loss:** This part is common to Joint-Embedding methods (like the one used in Cell 3). It encourages the predictor network to correctly predict the latent representation of a target view ($\mathbf{z}_{\text{target}}$) based on an input view ($\mathbf{z}_{\text{input}}$).
    * **$\mathbf{SIGReg}$ Loss:** This is the novel component: the **S**ketched **I**sotropic **G**aussian **Reg**ularization loss.

* ### **2. The Function of $\mathbf{SIGReg}$**
    The $\mathbf{SIGReg}$ loss is what makes LEJEPA a **mathematically rigorous** architecture. It serves to:
    * **Prevent Collapse:** It guarantees that the learned latent representations do not collapse into a trivial, single point, which is a major challenge in self-supervised learning.
    * **Enforce Structure:** It enforces the latent distribution to be the statistically optimal **Isotropic Gaussian distribution** ($N(0, I)$).

In essence, LEJEPA doesn't invent a new neural network architecture but provides a **provably optimal loss function** that simplifies training and ensures the resulting feature space is stable and high-quality for downstream tasks (like the causal planning in Cell 4).

Yes, I can absolutely compare LEJEPA's new loss calculation against the older, heuristic-driven methods it aims to replace.

---

## ⚖️ LEJEPA Loss vs. Older Self-Supervised Losses

The primary significance of LEJEPA (Lean Joint-Embedding Predictive Architecture) is its ability to eliminate complexity and instability by introducing a mathematically grounded regularization term.

| Feature | LEJEPA (New Loss) | Older Self-Supervised Methods (e.g., MoCo, SimSiam, DINO) |
| :--- | :--- | :--- |
| **Core Objective** | $\text{Prediction Loss} + \mathbf{SIGReg}$ (Isotropic Gaussian Regularization) | $\text{Prediction/Contrastive Loss}$ (often with hard coding to prevent collapse) |
| **Collapse Prevention** | **Guaranteed** by the **SIGReg** loss, which explicitly enforces the optimal $N(0, I)$ distribution on the latent embeddings. | Relied on complex **heuristics** (tricks) to prevent collapse, such as negative sampling, stop-gradients, or momentum encoders. |
| **Training Stability** | **High stability** and robustness across different hyperparameters (like batch size and $\lambda$). | **Low stability**, often requiring fine-tuning of momentum coefficients, learning rate schedules, and other hand-tuned parameters. |
| **Architecture** | Architecturally simple: just an **Encoder** and a **Predictor** network. | Often required complex dual-network architectures (e.g., **Teacher/Student** or **Momentum Encoders**) to function. |
| **Complexity** | **Single, simple trade-off hyperparameter** ($\lambda$). | Many ad-hoc hyperparameters required for each stabilization trick. |

***

## 💡 The Role of SIGReg

The innovation is entirely within the $\mathbf{SIGReg}$ (Sketched Isotropic Gaussian Regularization) component.

### The Problem in Old Methods: Collapse
Older methods primarily rely on making two different views of the same image/video (e.g., a zoomed crop and a colored crop) map to similar latent representations. This setup is inherently unstable because the simplest solution is for the encoder to map *all* inputs to the exact same vector (the **trivial/collapsing solution**).

### The Solution in LEJEPA: Optimal Constraint
LEJEPA solves this by providing the mathematical proof that the optimal distribution for the latent codes is the **Isotropic Gaussian** ($N(0, I)$).

1.  **Old Methods' Solution:** They added hacks like momentum encoders or stop-gradients to prevent the model from finding the collapsing solution, but these methods lacked theoretical justification and required constant tuning.
2.  **LEJEPA's Solution:** It adds the $\mathbf{SIGReg}$ loss, which **explicitly penalizes** the encoder if its output vectors deviate from the $N(0, I)$ distribution. This is a mathematically justified constraint that removes the need for all the clumsy heuristics, leading to a leaner and more robust framework.

In [22]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
from torch import optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from tqdm import tqdm
import random
import logging
import psutil
import gc

# Set logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s:%(message)s')

# Constants
STATE_DIM = 4      # Lat, Lon, Altitude, Speed
LATENT_DIM = 16
ACTION_DIM = 8
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- LEJEPA-inspired Isotropic Gaussian Regularization (SIGReg Proxy) ---
class IsotropicGaussianLoss(nn.Module):
    """
    A simplified proxy for LEJEPA's SIGReg (Sketched Isotropic Gaussian Regularization).
    It penalizes deviation of the batch statistics from the optimal N(0, I) distribution.
    The goal is to prevent representational collapse by enforcing zero mean and unit variance.
    """
    def __init__(self, lambda_reg=5.0): # lambda_reg is the trade-off hyperparameter
        super().__init__()
        self.lambda_reg = lambda_reg

    def forward(self, z):
        # 1. Zero-mean penalty (L2 norm of the batch mean)
        batch_mean = z.mean(dim=0)
        mean_penalty = torch.norm(batch_mean)

        # 2. Unit-variance penalty (Deviation of batch standard deviation from 1.0)
        batch_std = z.std(dim=0)
        std_penalty = torch.norm(batch_std - 1.0)

        # The total loss is a weighted sum of deviation from target mean (0) and std (1)
        total_regularization_loss = (mean_penalty + std_penalty) * self.lambda_reg
        return total_regularization_loss, mean_penalty.item(), std_penalty.item()

# Latent Projector (maps 4D state to 16D latent)
class LatentProjector(nn.Module):
    def __init__(self, state_dim=STATE_DIM, latent_dim=LATENT_DIM):
        super().__init__()
        self.encoder_net = nn.Sequential(
            nn.Linear(state_dim, 64),
            nn.ReLU(),
            nn.Linear(64, latent_dim)
        )

    def forward(self, x):
        if x.dim() == 1:
            x = x.unsqueeze(0)
        return self.encoder_net(x)

latent_projector = LatentProjector(state_dim=STATE_DIM, latent_dim=LATENT_DIM).to(device)
latent_projector.eval()
print("LatentProjector instance:", latent_projector)

# Latent Dynamics Predictor
class LatentDynamicsPredictor(nn.Module):
    def __init__(self, latent_dim=LATENT_DIM, action_dim=ACTION_DIM):
        super().__init__()
        self.fc1 = nn.Linear(latent_dim + action_dim, 64)
        self.fc2 = nn.Linear(64, latent_dim)

    def forward(self, latent, action):
        x = torch.cat([latent, action], dim=-1)
        x = torch.relu(self.fc1(x))
        z_tp1 = self.fc2(x)
        return z_tp1

predictor = LatentDynamicsPredictor(latent_dim=LATENT_DIM, action_dim=ACTION_DIM).to(device)
# Optimizer for both the predictor and the projector
optimizer_pldm = optim.Adam(list(predictor.parameters()) + list(latent_projector.parameters()), lr=1e-3)
print("Predictor instance:", predictor)

# Training Function (Modified to include LEJEPA Loss)
def train_latent_dynamics_model(predictor_model, latent_projector_model, optimizer, training_data_loader, epochs=5):
    predictor_model.train()
    latent_projector_model.train()

    # JEPA Predictive Loss
    predictive_criterion = torch.nn.functional.mse_loss

    # LEJEPA Regularization Loss (SIGReg Proxy)
    sigreg_criterion = IsotropicGaussianLoss(lambda_reg=5.0)

    print("\n--- Training Latent Dynamics Predictor with LEJEPA-inspired SIGReg Regularization ---")
    for epoch in range(epochs):
        total_loss = 0
        total_mean_pen = 0
        total_std_pen = 0
        for batch_idx, (latent_s_t, action_t, latent_s_t_plus_1) in tqdm(\
            enumerate(training_data_loader),\
            total=len(training_data_loader),\
            desc=f"Epoch {epoch+1}/{epochs}"
        ):
            latent_s_t, action_t, latent_s_t_plus_1 = (\
                latent_s_t.to(device),\
                action_t.to(device),\
                latent_s_t_plus_1.to(device)\
            )

            # 1. JEPA Predictive Step (Minimize prediction error)
            predicted_z_t_plus_1 = predictor_model(latent_s_t, action_t)
            predictive_loss = predictive_criterion(predicted_z_t_plus_1, latent_s_t_plus_1)

            # 2. LEJEPA Regularization Step (Enforce N(0, I) on latent codes)
            # Regularize the predicted state
            sigreg_loss_pred, mean_pen_pred, std_pen_pred = sigreg_criterion(predicted_z_t_plus_1)
            # Regularize the input state (enforcing the entire latent space distribution)
            sigreg_loss_input, mean_pen_input, std_pen_input = sigreg_criterion(latent_s_t)

            # Combine all loss components: JEPA Predictive Loss + LEJEPA Regularization Loss
            total_loss_batch = predictive_loss + sigreg_loss_pred + sigreg_loss_input

            optimizer.zero_grad()
            total_loss_batch.backward()
            optimizer.step()

            total_loss += total_loss_batch.item()
            total_mean_pen += (mean_pen_pred + mean_pen_input) / 2
            total_std_pen += (std_pen_pred + std_pen_input) / 2

        avg_loss = total_loss / len(training_data_loader)
        avg_mean_pen = total_mean_pen / len(training_data_loader)
        avg_std_pen = total_std_pen / len(training_data_loader)

        print(f"Epoch {epoch+1}/{epochs}, Total Avg Loss: {avg_loss:.6f}, Pred Loss: {predictive_loss.item():.6f}, SIGReg Mean Pen: {avg_mean_pen:.6f}, SIGReg Std Pen: {avg_std_pen:.6f}")
    print("--- Training Complete with LEJEPA-inspired Regularization ---")

# Data Loading Function (Uses LatentProjector from THIS cell)
def load_real_dynamics_data(device, adsb_dir="/content/adsb/kbtp/raw/2022", num_trajectories=10, max_traj_per_file=100):
    data = []
    trajectory_files = []
    # [... File loading and processing logic remains the same ...]
    try:
        for root, _, files in os.walk(adsb_dir):
            for file in files:
                if file.endswith('.csv'):
                    trajectory_files.append(os.path.join(root, file))
    except Exception as e:
        logging.critical(f"Failed to walk directory {adsb_dir}. Error: {e}")
        return []

    if not trajectory_files:
        logging.warning(f"No CSV files found in {adsb_dir}. Falling back to synthetic data.")
        return []

    print(f"\nAttempting to load REAL ADS-B data...")
    print(f"Found {len(trajectory_files)} CSV files. Limiting to {num_trajectories} files for demo speed.")

    trajectory_files = trajectory_files[:num_trajectories]

    for file in tqdm(trajectory_files, desc="Processing Trajectories"):
        try:
            if os.path.getsize(file) == 0:
                logging.warning(f"Skipping {file}: Empty file.")
                continue

            df = pd.read_csv(file, on_bad_lines='skip', engine='python')
            required_cols = ['Lat', 'Lon', 'Altitude', 'Speed']
            time_col = 'Time'

            available_cols = df.columns.tolist()
            if not all(col in available_cols for col in required_cols):
                logging.warning(f"Skipping {file}: Missing columns. Required: {required_cols}, Available: {available_cols}")
                continue

            for col in required_cols:
                df[col] = pd.to_numeric(df[col], errors='coerce')

            df.dropna(subset=required_cols, inplace=True)
            if df.empty or len(df) < 2:
                logging.warning(f"Skipping {file}: Fewer than 2 valid rows after cleaning.")
                continue

            # Clean Date and Time
            def clean_date(x):
                try:
                    if isinstance(x, (list, tuple)):
                        return '-'.join(str(i).zfill(2) if idx > 0 else str(i) for idx, i in enumerate(x))
                    if isinstance(x, str):
                        if x.startswith('['):
                            x = eval(x)
                            return '-'.join(str(i).zfill(2) if idx > 0 else str(i) for idx, i in enumerate(x))
                        if '-' in x or '/' in x:
                            return pd.to_datetime(x, errors='coerce').strftime('%Y-%m-%d')
                        return x
                    return str(x)
                except Exception as e:
                    return 'unknown'

            def clean_time(x):
                try:
                    if isinstance(x, (list, tuple)):
                        return ':'.join(str(i).zfill(2) for i in x)
                    if isinstance(x, str):
                        if x.startswith('['):
                            x = eval(x)
                            return ':'.join(str(i).zfill(2) for i in x)
                        if ':' in x:
                            return x
                        return x
                    return str(x)
                except Exception as e:
                    return '00:00:00'

            if 'Date' in df.columns:
                df['Date'] = df['Date'].apply(clean_date)
            if 'Time' in df.columns:
                df['Time'] = df['Time'].apply(clean_time)
                df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S', errors='coerce')

            if 'Tail' in df.columns and 'Date' in df.columns:
                df['traj_id'] = df['Tail'].astype(str) + '_' + df['Date'].astype(str)
                df['traj_id'] = df['traj_id'].fillna('missing_' + pd.Series(range(len(df)), index=df.index).astype(str))
            else:
                logging.warning(f"Missing 'Tail' or 'Date' in {file}. Using index as traj_id.")
                df['traj_id'] = pd.Series(range(len(df)), index=df.index).astype(str)


            traj_ids = df['traj_id'].unique()[:max_traj_per_file]

            latent_projector.eval()

            for traj_id in traj_ids:
                traj = df[df['traj_id'] == traj_id].sort_values(time_col)
                if len(traj) < 2:
                    logging.warning(f"Skipping trajectory {traj_id} in {file}: Fewer than 2 points.")
                    continue

                for i in range(len(traj) - 1):
                    try:
                        row_t = traj.iloc[i][required_cols]
                        row_tp1 = traj.iloc[i+1][required_cols]
                        if not all(row_t.apply(lambda x: isinstance(x, (int, float)) and not pd.isna(x))) or \
                           not all(row_tp1.apply(lambda x: isinstance(x, (int, float)) and not pd.isna(x))):
                            logging.warning(f"Non-numeric data in trajectory {traj_id} at index {i}: {row_t.values}, {row_tp1.values}")
                            continue
                        state_t = torch.tensor(row_t.values.astype(np.float64), dtype=torch.float32, device=device)
                        state_tp1 = torch.tensor(row_tp1.values.astype(np.float64), dtype=torch.float32, device=device)
                        action = torch.tensor([\
                            state_tp1[3] - state_t[3],  # Delta Speed
                            state_tp1[2] - state_t[2],  # Delta Altitude
                            0.0, 0.0, 0.0, 0.0, 0.0, 0.0\
                        ], dtype=torch.float32, device=device)
                        with torch.no_grad():
                            projected_state_t = latent_projector(state_t).squeeze(0).cpu()
                            projected_state_tp1 = latent_projector(state_tp1).squeeze(0).cpu()
                        data.append((projected_state_t, action.cpu(), projected_state_tp1))
                    except Exception as e:
                        logging.warning(f"Error processing trajectory {traj_id} in {file}: {e}")

            del df
            gc.collect()
        except Exception as e:
            logging.warning(f"Error reading {file}: {e}")

    print(f"Loaded {len(data)} real dynamics samples from {len(trajectory_files)} files.")
    return data

# Synthetic Data Fallback
def generate_synthetic_data(num_trajectories=1000, trajectory_length=20):
    latent_projector.eval()
    synthetic_data = []
    # [.. Synthetic data generation logic remains the same ...]
    for _ in range(num_trajectories):
        base_state = torch.tensor([45.47, -73.74, 37000.0, 490.0], dtype=torch.float32, device=device)
        base_state += torch.randn(STATE_DIM, device=device) * 0.1
        current_state = base_state.clone()
        for _ in range(trajectory_length):
            delta_v = random.uniform(-10.0, 10.0)
            delta_alt = random.uniform(-50.0, 50.0)
            action = torch.tensor([delta_v, delta_alt, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],\
                                  dtype=torch.float32, device='cpu').detach()
            next_state = current_state.clone()
            next_state[3] += delta_v
            next_state[2] += delta_alt
            with torch.no_grad():
                projected_state_t = latent_projector(current_state.to(device)).squeeze(0).cpu().detach()
                projected_state_tp1 = latent_projector(next_state.to(device)).squeeze(0).cpu().detach()
            synthetic_data.append((projected_state_t, action, projected_state_tp1))
            current_state = next_state.clone()
    return synthetic_data

# Execution Block
FILE_LIMIT = 20
print("\n--- Starting AGI Demo Pipeline (Cell 3) ---\n")
dynamics_training_data = load_real_dynamics_data(device, num_trajectories=FILE_LIMIT, max_traj_per_file=100)
if not dynamics_training_data:
    print(f"--- CRITICAL WARNING: NO REAL DATA LOADED from {FILE_LIMIT} files. CREATING SYNTHETIC FALLBACK. ---\n")
    dynamics_training_data = generate_synthetic_data(num_trajectories=1000)
    print(f"Successfully generated {len(dynamics_training_data)} synthetic causal samples for demonstration.")
if dynamics_training_data:
    print("\nPreparing DataLoader for batch training...")
    z_t_list, a_t_list, z_tp1_list = zip(*dynamics_training_data)
    Z_T = torch.stack(z_t_list)
    A_T = torch.stack(a_t_list)
    Z_TP1 = torch.stack(z_tp1_list)
    dataset = TensorDataset(Z_T, A_T, Z_TP1)
    BATCH_SIZE = 64
    dynamics_dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
    print(f"Total batches for training: {len(dynamics_dataloader)}")
    train_latent_dynamics_model(predictor, latent_projector, optimizer_pldm, dynamics_dataloader, epochs=5)
else:
    print("FATAL ERROR: No data (real or synthetic) could be prepared. Training aborted.")
print("\nCell 3 execution complete.")
print("Cell 3 completed. Predictor and latent_projector defined, incorporating LEJEPA-inspired regularization.")

Using device: cuda
LatentProjector instance: LatentProjector(
  (encoder_net): Sequential(
    (0): Linear(in_features=4, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=16, bias=True)
  )
)
Predictor instance: LatentDynamicsPredictor(
  (fc1): Linear(in_features=24, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=16, bias=True)
)

--- Starting AGI Demo Pipeline (Cell 3) ---


Attempting to load REAL ADS-B data...
Found 246 CSV files. Limiting to 20 files for demo speed.


Processing Trajectories: 100%|██████████| 20/20 [18:10<00:00, 54.52s/it]


Loaded 588006 real dynamics samples from 20 files.

Preparing DataLoader for batch training...
Total batches for training: 9188

--- Training Latent Dynamics Predictor with LEJEPA-inspired SIGReg Regularization ---


Epoch 1/5: 100%|██████████| 9188/9188 [00:25<00:00, 365.65it/s]


Epoch 1/5, Total Avg Loss: 72044.490055, Pred Loss: 68.497559, SIGReg Mean Pen: 3069.874507, SIGReg Std Pen: 3703.850365


...

Epoch 3/5, Total Avg Loss: 68078.858114, Pred Loss: 98.181709, SIGReg Mean Pen: 3074.741265, SIGReg Std Pen: 3710.518212


Epoch 4/5: 100%|██████████| 9188/9188 [00:24<00:00, 369.37it/s]


Epoch 4/5, Total Avg Loss: 68070.477283, Pred Loss: 198.857910, SIGReg Mean Pen: 3074.727989, SIGReg Std Pen: 3710.064654


Epoch 5/5: 100%|██████████| 9188/9188 [00:24<00:00, 373.63it/s]

Epoch 5/5, Total Avg Loss: 68064.360929, Pred Loss: 160.120453, SIGReg Mean Pen: 3074.834243, SIGReg Std Pen: 3709.657837
--- Training Complete with LEJEPA-inspired Regularization ---

Cell 3 execution complete.
Cell 3 completed. Predictor and latent_projector defined, incorporating LEJEPA-inspired regularization.


## CEL4-NEW

In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import pandas as pd
import numpy as np
from transformers import CLIPProcessor, CLIPModel
import gc
import random
import logging
import pytz
import datetime
from warnings import filterwarnings
filterwarnings("ignore")

# --- CONTEXTUAL CONSTANTS (Consistent with Cell 3) ---
LATENT_DIM = 16
ACTION_DIM = 8
CLIP_FEATURE_DIM = 512 # Input dimension for visual features
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

AIRPORTS = {
    "CYUL": {"lat": 45.4706, "lon": -73.7408, "name": "Montreal-Trudeau International"},
    "LFPG": {"lat": 49.0128, "lon": 2.5500, "name": "Paris-Charles de Gaulle"}
}
AIRCRAFT_PERFORMANCE = {
    "Boeing777_300ER": {
        "max_speed_kts": 490.0,
        "cruise_altitude_ft": 37000.0,
        "range_nm": 7370.0
    }
}


# --- CLASS DEFINITIONS (Re-using LEJEPA/JEPA structure from Cell 3) ---

class LatentDynamicsPredictor(nn.Module):
    """Predicts the next latent state (z_t+1) given current state (z_t) and action (a_t)."""
    def __init__(self, latent_dim=LATENT_DIM, action_dim=ACTION_DIM):
        super().__init__()
        self.fc1 = nn.Linear(latent_dim + action_dim, 64)
        self.fc2 = nn.Linear(64, latent_dim)

    def forward(self, latent, action):
        x = torch.cat([latent, action], dim=-1)
        x = torch.relu(self.fc1(x))
        z_tp1 = self.fc2(x)
        return z_tp1

class LatentProjector(nn.Module):
    """Maps the input feature dimension (visual, 512D) to the LEJEPA-stabilized latent space (16D)."""
    def __init__(self, state_dim=CLIP_FEATURE_DIM, latent_dim=LATENT_DIM):
        super().__init__()
        self.encoder_net = nn.Sequential(
            nn.Linear(state_dim, 64),
            nn.ReLU(),
            nn.Linear(64, latent_dim)
        )
    def forward(self, x):
        if x.dim() == 1:
            x = x.unsqueeze(0)
        return self.encoder_net(x)

# --- HELPER FUNCTION FOR VISUAL FEATURE EXTRACTION (CLIP) ---

def load_and_process_video(video_path, processor, model, device, num_frames=1):
    try:
        from torchvision.io import read_video
        video, _, _ = read_video(video_path, pts_unit='sec')
        video = video[:num_frames].to(device)
        features_list = []
        for frame in video:
            inputs = processor(images=frame.unsqueeze(0), return_tensors="pt").to(device)
            with torch.no_grad():
                features = model.get_image_features(**inputs)
                features_list.append(features)

        if not features_list:
             return None, "No frames processed"

        averaged_features = torch.mean(torch.stack(features_list), dim=0)
        del features_list, features, inputs, video, frame
        torch.cuda.empty_cache()
        gc.collect()

        return averaged_features, None

    except Exception as e:
        return None, str(e)

# --- MODEL INSTANTIATION (Conceptual Load of Trained Weights) ---

# Load CLIP models for visual features (planning targets)
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32", use_fast=False)

# Instantiate the planning models (conceptually carrying weights from Cell 3)
latent_projector_planning = LatentProjector(state_dim=CLIP_FEATURE_DIM, latent_dim=LATENT_DIM).to(device)
predictor_planning = LatentDynamicsPredictor(latent_dim=LATENT_DIM, action_dim=ACTION_DIM).to(device)


# --- CORE PLANNING FUNCTION ---

def plan_montreal_to_paris_flight(start_airport_data, target_airport_data, aircraft_model_data,\
                                  encoder_model, processor_instance, predictor_model, latent_projector_instance,\
                                  planning_horizon=50, action_dim=ACTION_DIM, num_action_samples=50):
    encoder_model.eval()
    predictor_model.eval()
    latent_projector_instance.eval()

    # --- 1. SETTING VISUAL START/TARGET IN LEJEPA LATENT SPACE ---
    initial_video_path = '/content/gdrive/MyDrive/datasets/TartanAviation/vision/1_2023-02-22-15-21-49/1_2023-02-22-15-21-49.mp4'
    target_video_path = initial_video_path # Using same video conceptually

    initial_features, _ = load_and_process_video(initial_video_path, processor_instance, encoder_model, device, num_frames=1)
    target_features, _ = load_and_process_video(target_video_path, processor_instance, encoder_model, device, num_frames=1)

    if initial_features is None or target_features is None:
        print("WARNING: Video load failed. Using dummy features for planning targets.")
        dummy_feature_shape = (1, CLIP_FEATURE_DIM)
        initial_features = torch.rand(dummy_feature_shape).to(device)
        target_features = torch.rand(dummy_feature_shape).to(device)

    # Project Visual Features to the 16D LATENT SPACE
    with torch.no_grad():
        current_latent_state = latent_projector_instance(initial_features)
        target_latent_state = latent_projector_instance(target_features)

    del initial_features, target_features
    torch.cuda.empty_cache()
    gc.collect()

    # --- 2. CONTEXT AND ETHICAL BOUNDARIES ---
    ETHICAL_BOUNDARY_LATENT_VECTOR = torch.zeros(1, LATENT_DIM).to(device)
    # Simulate dynamic salience (e.g., poor weather/visibility)
    salience = torch.rand(1).to(device) * 0.8

    print("\n--- Starting Causal Flight Plan (MPPI) ---")
    print(f"Current Latent State Shape: {current_latent_state.shape} (LEJEPA-stabilized latent space)")
    print(f"Planning Horizon: {planning_horizon} steps.")

    best_action_sequence = []
    current_latent = current_latent_state

    # --- 3. ITERATIVE PLANNING LOOP ---
    for step in range(planning_horizon):
        # Generate candidate actions
        candidate_actions = torch.rand(num_action_samples, action_dim).to(device) * 2.0 - 1.0

        repeated_current_latent = current_latent.repeat(num_action_samples, 1)

        with torch.no_grad():
            # Use the LEJEPA-trained PREDICTOR to simulate the next state
            simulated_next_latents = predictor_model(repeated_current_latent, candidate_actions)

        del repeated_current_latent

        simulated_trajectories_cost = []
        for i in range(num_action_samples):
            simulated_next_latent = simulated_next_latents[i].unsqueeze(0)

            # --- COST FUNCTION (Incorporating AGI Pillars) ---
            # 1. Goal Proximity
            goal_proximity_cost = torch.norm(target_latent_state - simulated_next_latent) * 1.0
            # 2. Conceptual Fuel Cost
            conceptual_fuel_cost = torch.norm(candidate_actions[i]) * 0.05
            # 3. Ethical/Safety Cost (Penalty for deviating from safety zone)
            ethical_cost = 5.0 * torch.norm(ETHICAL_BOUNDARY_LATENT_VECTOR - simulated_next_latent)
            # 4. Salience Alignment Cost (Cautious penalty)
            salience_alignment_cost = 2.0 * torch.norm(candidate_actions[i]) * salience

            total_cost = goal_proximity_cost + conceptual_fuel_cost + ethical_cost + salience_alignment_cost
            simulated_trajectories_cost.append(total_cost.item())

        del simulated_next_latents
        torch.cuda.empty_cache()
        gc.collect()

        best_candidate_idx = torch.argmin(torch.tensor(simulated_trajectories_cost))

        # Get the optimal action
        optimal_action_for_step = candidate_actions[best_candidate_idx]

        best_action_sequence.append(optimal_action_for_step.squeeze().cpu().numpy())

        with torch.no_grad():
            # Update current_latent using the chosen optimal action (Causal step)
            optimal_action_input = optimal_action_for_step.unsqueeze(0)
            current_latent = predictor_model(current_latent, optimal_action_input)

        del optimal_action_for_step, optimal_action_input, candidate_actions
        torch.cuda.empty_cache()
        gc.collect()


    print(f"\nConceptual Flight Plan generated for {planning_horizon} steps (first 5 actions shown):")
    for i, action in enumerate(best_action_sequence[:5]):
        print(f"Step {i+1}: {np.round(action, 4)}")
    return best_action_sequence

# --- EXECUTION OF CELL 4 ---
print(f"Using device: {device}")
print("\n--- Running Cell 4: Causal Planning ---")
try:
    conceptual_flight_plan_actions = plan_montreal_to_paris_flight(
        AIRPORTS["CYUL"], AIRPORTS["LFPG"], AIRCRAFT_PERFORMANCE["Boeing777_300ER"],
        model, processor, predictor_planning, latent_projector_planning,
        num_action_samples=50
    )
    print("\nCell 4 execution complete. The final action sequence represents the optimal planned flight path.")
except Exception as e:
    print(f"\nAn error occurred during final flight planning: {e}")
    print("Execution aborted.")

Using device: cuda

--- Running Cell 4: Causal Planning ---

--- Starting Causal Flight Plan (MPPI) ---
Current Latent State Shape: torch.Size([1, 16]) (LEJEPA-stabilized latent space)
Planning Horizon: 50 steps.

Conceptual Flight Plan generated for 50 steps (first 5 actions shown):
Step 1: [ 0.1126 -0.1295  0.5835  0.1665  0.0507 -0.2585 -0.1045  0.557 ]
Step 2: [-0.2135  0.3874  0.0369  0.0562 -0.7279 -0.3626 -0.1272 -0.2128]
Step 3: [-0.0016  0.5347 -0.004  -0.5805  0.683  -0.6664  0.3506  0.081 ]
Step 4: [-0.1117  0.4102  0.1384  0.2543  0.2076 -0.216   0.552   0.2985]
Step 5: [-0.3976 -0.0748 -0.966   0.271  -0.0294  0.0108  0.3414 -0.2553]

Cell 4 execution complete. The final action sequence represents the optimal planned flight path.
